In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [4]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()

    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return x

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [25]:
# Hyperparameters

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 50

In [18]:
train_data = datasets.MNIST(root="dataset/", train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_data, 
                          batch_size=batch_size, 
                          shuffle=True)

test_data = datasets.MNIST(root="dataset/", train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_data, 
                          batch_size=batch_size, 
                          shuffle=True)



In [19]:
model = NN(input_size=input_size, num_classes=num_classes)

In [20]:
model.to(device)

NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [22]:
criterion = nn.CrossEntropyLoss()

In [23]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0], -1) # 28x28x1 => 784

    # Forward Pass
    scores = model(data)
    loss = criterion(scores, targets)
    
    # Backward Pass
    optimizer.zero_grad()
    loss.backward()

    # Gradient Descent
    optimizer.step()

  print(f"[*] Epoch {epoch}: loss = {loss}")

[*] Epoch 0: loss = 0.5493718385696411
[*] Epoch 1: loss = 0.5389233231544495
[*] Epoch 2: loss = 0.1990954577922821
[*] Epoch 3: loss = 0.15325486660003662
[*] Epoch 4: loss = 0.30082038044929504
[*] Epoch 5: loss = 0.3804765045642853
[*] Epoch 6: loss = 0.3342514932155609
[*] Epoch 7: loss = 0.44302257895469666
[*] Epoch 8: loss = 0.1958870142698288
[*] Epoch 9: loss = 0.2080163061618805
[*] Epoch 10: loss = 0.2973130941390991
[*] Epoch 11: loss = 0.21885040402412415
[*] Epoch 12: loss = 0.15559057891368866
[*] Epoch 13: loss = 0.4417840242385864
[*] Epoch 14: loss = 0.37429484724998474
[*] Epoch 15: loss = 0.18886396288871765
[*] Epoch 16: loss = 0.2993009090423584
[*] Epoch 17: loss = 0.2255537360906601
[*] Epoch 18: loss = 0.3728806674480438
[*] Epoch 19: loss = 0.379336953163147
[*] Epoch 20: loss = 0.26734426617622375
[*] Epoch 21: loss = 0.364096462726593
[*] Epoch 22: loss = 0.29750820994377136
[*] Epoch 23: loss = 0.07283991575241089
[*] Epoch 24: loss = 0.1525401920080185
[*

In [26]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking Accuracy on training data")
  else:
    print("Checking Accuracy on testing data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    acc = float(num_correct)/float(num_samples)*100
    print(f"Got {num_correct} / {num_samples} with accuracy {acc:.2f}")
  model.train()
  return acc

In [28]:
check_accuracy(train_loader, model)

Checking Accuracy on training data
Got 59435 / 60000 with accuracy 99.06


99.05833333333334

In [29]:
check_accuracy(test_loader, model)

Checking Accuracy on testing data
Got 9654 / 10000 with accuracy 96.54


96.54